In [1]:
# install and import package
import subprocess
def run_cmd(cmd):
    return subprocess.getoutput(cmd)
# !pip install -q transformers==4.50 sentencepiece 
# !pip install -q accelerate datasets==3.5 rouge_score bert-score nltk
# !pip install -U -q sentence-transformers evaluate faiss-cpu
# !pip install -q --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib google-auth
# !pip install -q --upgrade cffi
#!pip install --upgrade ipywidgets

print('\n' + "="*20 + " PWD " + "="*20)
!ls -la

print('\n' + "="*20 + " IMPORT AND DOWNLOAD " + "="*20)
import faiss, logging, json, mimetypes, transformers, io
from email.message import EmailMessage
from datasets import load_dataset, Dataset
from bert_score import score as bertscores
from transformers import MBartForConditionalGeneration, BartForConditionalGeneration, AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import DataCollatorForSeq2Seq, get_scheduler, MBartForConditionalGeneration, Seq2SeqTrainer, T5Tokenizer, T5ForConditionalGeneration
from transformers import get_scheduler, DataCollatorWithPadding, TrainingArguments, Seq2SeqTrainingArguments
from subprocess import call
from IPython.display import display
import shutil, os, sys, torch, os.path, ssl, smtplib, re, random, time, torch
import pprint as pp
import pandas as pd
import numpy as np
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt
import nltk, evaluate
from evaluate import load
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from accelerate import Accelerator
from torch.optim import AdamW
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from googleapiclient.errors import HttpError
#nltk.download("punkt")
nltk.download('punkt_tab')
pd.options.mode.copy_on_write = False
from matplotlib import pyplot as plt
pd.set_option('display.max_colwidth', None)

print("="*20 + " GPU " + "="*20)
if 'not found' not in run_cmd('nvidia-smi'):
    !nvidia-smi
    print()
    print('__Python VERSION:', sys.version)
    print('__pyTorch VERSION:', torch.__version__)
    print('__CUDA VERSION')
    ! nvcc --version
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__Devices')
    call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
    print('Active CUDA Device: GPU', torch.cuda.current_device())
    print ('Available devices ', torch.cuda.device_count())
    print ('Current cuda device ', torch.cuda.current_device())
else:
    print('NON-GPU VERSION')

print('\n' + "="*20 + " OS " + "="*20)
!cat /etc/*release

print('\n' + "="*20 + " PYTHON " + "="*20)
!python --version
!pip freeze > requirements.txt 
print("Check output folder for requirements.txt")



==================== PWD ====================
total 728
drwxrwxr-x  8 npl npl   8192 Jul  6 17:51 .
drwxrwxr-x 16 npl npl   8192 Jul  5 05:35 ..
drwxrwxr-x  3 npl npl    152 Jul  4 06:32 .working
drwxrwxr-x  2 npl npl   8192 Jul  4 06:32 __pycache__
-rw-rw-r--  1 npl npl 222355 Jul  5 04:52 active-learning-main-run.ipynb
-rw-rw-r--  1 npl npl 272758 Jul  6 12:35 app.log
-rw-rw-r--  1 npl npl      0 Jul  6 05:10 app_critical.log
drwxrwxr-x  2 npl npl    152 Jul  4 06:32 bertscore
drwxrwxr-x  2 npl npl   8192 Jul  4 06:32 bleu
-rw-rw-r--  1 npl npl   9909 Jul  4 06:32 drive_func.py
-rw-rw-r--  1 npl npl  13861 Jul  4 06:32 mainrun.py
-rw-rw-r--  1 npl npl   6197 Jul  6 05:09 requirements.txt
-rw-rw-r--  1 npl npl  47720 Jul  6 12:26 result.csv
drwxrwxr-x  2 npl npl    152 Jul  4 06:32 rouge
-rw-rw-r--  1 npl npl   1139 Jul  4 06:32 run.slurm
-rw-rw-r--  1 npl npl   6938 Jul  4 06:32 run_vit5_random.out
-rw-rw-r--  1 npl npl   5798 Jul  4 06:32 select_algorithm.py
-rw-rw-r--  1 npl npl  

[nltk_data] Downloading package punkt_tab to /home/npl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


==================== GPU ====================
Sun Jul  6 18:03:29 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   38C    P0             245W / 400W |  51409MiB / 81920MiB |     99%      Default |
|                                         |                      |             Disabled |
+---------------------

In [2]:
# setup logging
def show_only_critical(record):
    return record.levelname == "CRITICAL"
def initialize_logging(name): 
    logger = logging.getLogger(name)
    console_handler = logging.StreamHandler()
    file_handler = logging.FileHandler("app.log", mode = "a", encoding="utf-16")
    file_critical_handler = logging.FileHandler("app_critical.log", mode = "a", encoding="utf-16")
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    logger.addHandler(file_critical_handler)
    formatter = logging.Formatter(
        "[{asctime}.{msecs:0<3.0f} - {levelname}]  {message}",
        style = "{",
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)
    file_critical_handler.setFormatter(formatter)
    logger.setLevel("DEBUG")
    console_handler.setLevel("DEBUG")
    file_handler.setLevel("INFO")
    file_critical_handler.addFilter(show_only_critical)
    logger.debug("initialized")
    return logger

In [3]:
#get number of word in string
def len_word(str):
    return len(re.split('[-\s_]', str))

#get index of nth-occur char or -1
def nth_occur_char(st, char=' ', nth=1, begin = 0):
    n = 0
    i = begin
    while i < len(st):
        i = st.find(' ', i,)
        if i == -1:
            return -1
        else:
            n += 1
            if n == nth:
                return i
        i+=1
    return -1

# clean string list - remove too short length string item, remove hyphense,...
def clean_list(lst):
    l, i = len(lst), 0
    while i < l:
        if len(lst[i]) <= 2:
            lst.remove(lst[i])
            l -= 1
        else:
            lst[i] = clean_text(lst[i])
            i += 1    
    return lst

#clean text
def clean_text(text):
    return re.sub(r'^\n|^\s|^-\s', '', text)

#preprocess data
def preprocess_dataset(data):
    data.set_format(type = "pandas")
    df = data[:]
    df_detail = pd.DataFrame(columns=["cluster_id", "title", "anchor_text", "raw_text"])
    df_cluster_info = pd.DataFrame(columns=["cluster_id", "summary", "category"])
    for i in range(len(df["single_documents"])): 
        temp = pd.DataFrame.from_dict(df["single_documents"][i].tolist(), orient='columns')
        temp["cluster_id"] = i
        df_detail = pd.concat([df_detail, temp], axis=0, )
        temp2 = df.iloc[[i]].copy()
        temp2["cluster_id"] = i
        temp2 = temp2.drop(["single_documents"], axis="columns")
        df_cluster_info = pd.concat([df_cluster_info, temp2], axis=0, )
    df_detail = df_detail.reset_index()
    df_detail = df_detail.drop("index", axis="columns")
    return df_detail, df_cluster_info

def preprocess_sentence_dataset(document_data):
    input_type = ""
    if isinstance(document_data, Dataset):
        input_type = "dataset"
        document_data.set_format('pandas')
        document_data = document_data[:]
    else:
        input_type = "pandas"
    document_data = document_data.copy()
    document_data.loc[:,'raw_text'] = document_data.loc[:,'raw_text'].str.split('\.(?!\d)|\u2026|\n|(?<!Ảnh)[:;] (?=[A-Z])|: (?=["“])')
    document_data.loc[:,'raw_text'].apply(lambda x: clean_list(x))
    document_data['raw_index'] = pd.Series( [[i for i in range(len(e))] for e in document_data['raw_text']],index = document_data.index)
    document_data = document_data.explode(['raw_text','raw_index']).reset_index(drop=True)

    if input_type == "dataset":
        document_data = Dataset.from_pandas(document_data)
    return document_data
    
def dataset_sentence_split(data, length_split):
    input_type = ""
    if isinstance(data, Dataset):
        input_type = "dataset"
        data.set_format('pandas')
        data = data[:]
    else:
        input_type = "pandas"

    data.loc[:,'raw_text'] = data.loc[:,'raw_text'].map(lambda x: x if nth_occur_char(x, ' ', length_split) == -1 else [x[:nth_occur_char(x, ' ', length_split)], x[nth_occur_char(x, ' ', length_split)+1:]])
    data = data.explode(['raw_text']).reset_index(drop=True)
    
    if input_type == "dataset":
        data = Dataset.from_pandas(data)
    return data 
def get_embeddings(
    data,
    embed_col,
    tokenizer,
    model,
    encoder_seq_length,
    device,
    return_device,
    graph_split = None, # truncate - None / sentence split to sentences
    graph_merge = None, # mean/max/None pooling
    sentence_split = None, # truncate - None / split to 2 sentence, default truncate
    sentence_merge = None, # mean/max/None pooling MOI
    sentence_vec = None, # cls - None/mean/max pooling MOI
    return_type = 'datasets',
    batch_size = 256
):
    
    if isinstance(data, pd.DataFrame) == True:
      data = Dataset.from_pandas(data)
        
    if "embeddings" in data.column_names:
        if return_type == 'pandas':
            data.set_format('pandas')
            return data[:]
        else: #if return_type == 'datasets':
            return data
            
    if graph_split == "sentence":
        data = preprocess_sentence_dataset(data)
        if sentence_split == 'split':
            data = dataset_sentence_split(data, 20)
            
    
    encoded_input = data.map(lambda x: tokenizer(x[embed_col], truncation=True, padding=True, max_length=encoder_seq_length, return_tensors="pt"), batched=True, remove_columns=data.column_names)
    dataloader_or_data = DataLoader(
        encoded_input,
        shuffle=False,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(
            tokenizer=tokenizer
        ),
        pin_memory=(str(device)),
    )
    if model.device != device:
        model2 = model.to(device)
        del model
        torch.cuda.empty_cache()
        model = model2
    
    num_obs = len(dataloader_or_data.dataset)
    dim = list(model.state_dict().items())[-1][1].shape[0]
    embeddings = torch.empty((num_obs, dim), dtype=torch.float, device=device)
    
    with torch.no_grad():
        model.eval()
        start = 0
        for batch in tqdm(dataloader_or_data, desc="Embeddings created"):
            batch = {
                k: v.to(device) for k, v in batch.items()
            }
            predictions = model(**batch)
    
            if sentence_vec == 'mean':
                batch_embeddings = torch.mean(predictions.last_hidden_state[:,1:,:], 1)
            elif sentence_vec == 'max':
                batch_embeddings = torch.max(predictions.last_hidden_state[:,1:,:], 1).values
            else: #cls
                batch_embeddings = predictions.last_hidden_state[:,0,:]
    
            end = start + len(batch["input_ids"])  # len(batch[list(batch.keys())[0]])
            embeddings[start:end].copy_(batch_embeddings, non_blocking=True)
            start = end
            del predictions, batch
        temp = embeddings.to(return_device)
        del embeddings
        torch.cuda.empty_cache()
        embeddings = temp
        data = data.add_column('embeddings',embeddings.tolist())
        data.set_format('pandas')
        data = data[:]
        if graph_split != None:
            if sentence_split == 'split': 
                if sentence_merge == "mean":
                    data = data.groupby(['cluster_id', 'title', 'anchor_text', 'raw_index']).agg({
                        'raw_text': lambda x : ' '.join(x),
                        'embeddings': 'mean'
                    }).reset_index()
                elif sentence_merge == "max":
                    data = data.groupby(['cluster_id', 'title', 'anchor_text', 'raw_index']).agg({
                        'raw_text': lambda x : ' '.join(x),
                        'embeddings': lambda x : np.max(list(x), axis=0)
                    }).reset_index()
            if graph_merge == "mean":
                data = data.drop('raw_index', axis=1)
                data = data.groupby(['cluster_id', 'title', 'anchor_text']).agg({
                    'raw_text': lambda x : '. '.join(x),
                    'embeddings': 'mean'
                }).reset_index()
            elif graph_merge == "max":
                data = data.drop('raw_index', axis=1)
                data = data.groupby(['cluster_id', 'title', 'anchor_text']).agg({
                    'raw_text': lambda x : '. '.join(x),
                    'embeddings': lambda x : np.max(list(x), axis=0)
                }).reset_index()
        if return_type == 'pandas':
            return data
        else: #if return_type == 'datasets':
            return Dataset.from_pandas(data)

In [4]:
# select algorithm
def average_similarity(embed, data_embeddings):
    if data_embeddings.size(0) == 0:  # Kiểm tra nếu data_embeddings trống
        return 0
    # Giả định sử dụng cosine similarity trên GPU
    cos = torch.nn.functional.cosine_similarity(embed.unsqueeze(0), data_embeddings)
    return cos.mean().item()  # Trả về giá trị trung bình của độ tương đồng
def select_idds_samples(unlabel_train_data, labeled_train_data, lamb, k, device='cuda'):
    
    # Kiểm tra và in kích thước embedding của từng DataFrame trước khi chuyển đổi sang tensor
    assert "embeddings" in unlabel_train_data.columns, "Column 'embeddings' missing in unlabel_train_data"
    assert "embeddings" in labeled_train_data.columns, "Column 'embeddings' missing in labeled_train_data"
    
    # Chuyển embeddings của unlabel và labeled thành tensor trên GPU
    unlabel_embeddings = torch.tensor(np.array(list(unlabel_train_data["embeddings"].values), dtype='float32'), device=device)
    labeled_embeddings = torch.tensor(np.array(list(labeled_train_data["embeddings"].values), dtype='float32'), device=device)
    
    # Tính toán độ tương đồng trên GPU cho toàn bộ unlabel_train_data
    similarities_unlabel = lamb * torch.tensor(
        [average_similarity(embed, unlabel_embeddings) for embed in unlabel_embeddings],
        device=device
    )
    
    # Tính toán độ tương đồng giữa unlabel_train_data và labeled_train_data trên GPU
    similarities_label = (1 - lamb) * torch.tensor(
        [average_similarity(embed, labeled_embeddings) for embed in unlabel_embeddings],
        device=device
    )
    
    # Tính điểm cho từng hàng
    scores = similarities_unlabel - similarities_label

    # Tìm chỉ số của điểm cao nhất
    if len(scores) < k:
        k = len(scores)
    k_max_idx = torch.topk(scores, k=k).indices.tolist()

    # Trả về hàng có điểm cao nhất (chuyển về CPU để lấy dữ liệu từ DataFrame)
    return unlabel_train_data.iloc[k_max_idx].copy()
    

def select_random_samples(unlabel_train_data, k):
    return unlabel_train_data.sample(n=k, random_state=random.randint(1, 100000), axis=0) 
    
def data_select( 
        unlabel_train_data, budget = -1,
        init = 100, k_init = 10, init_type = "random",
        iter = 10,  k_iter = -1, iter_type = "random",
        lamb = 0.67, device="cuda"
    ): 
    # k<= 0 boc het, 0<k <1, boc theo ti le, k>=1, boc theo so luong 
    # co the xet thaam so moi laan boc k
    # k=1 moi laan chi boóc mau: 1200 maau maat 160s
    # k=iter moi laan boóc iter: 1200 maau maat 4s
    if "iter" in unlabel_train_data.columns:
        return unlabel_train_data
    data_len = len(unlabel_train_data)
    if budget <= 0 or budget >= data_len:
        budget = data_len
    elif budget > 0 and budget < 1:
        budget = round(budget*data_len)
    elif budget >=1:
        budget = round(budget)
        
    if init > budget:
        init = budget
    if iter > budget:
        iter = budget
        
    if k_init <= 0 or k_init >= init:
        k_init = init
    elif k_init > 0 and k_init < 1:
        k_init = round(k_init*init)
    elif k_init >=1:
        k_init = round(k_init)

    if k_iter <= 0 or k_iter >= iter:
        k_iter = iter
    elif k_iter > 0 and k_iter < 1:
        k_iter = round(k_iter*iter)
    elif k_iter >=1:
        k_iter = round(k_iter)
        
    labeled_train_data = pd.DataFrame(columns=["cluster_id", "title", "anchor_text", "raw_text", "embeddings", "iter"])
    if len(unlabel_train_data) < init: 
        assert "Not enough data" == False
        return
    
    # Tạo một mask để đánh dấu các hàng đã chọn
    mask = pd.Series([False] * len(unlabel_train_data))
    progress_bar = tqdm(range(budget), desc=f"{iter_type} Select")

    
    if init_type == "random":
        available_indices = unlabel_train_data[~mask].index
        samples_index = np.random.choice(available_indices, size=init, replace=False)
        sample = unlabel_train_data.loc[samples_index].copy()
        mask[samples_index] = True
        labeled_train_data = pd.concat([labeled_train_data, sample], ignore_index=True)
    else:
        l = init
        while l > 0:
            if l < k_init:
                k_init = l
            sample = select_idds_samples(unlabel_train_data[~mask], labeled_train_data, lamb, k=k_init, device=device)            
            mask[sample.index.tolist()] = True
            labeled_train_data = pd.concat([labeled_train_data, sample], ignore_index=True)
            l = l - k_init
    labeled_train_data["iter"] = 0
    progress_bar.update(init)
    
    # Vòng lặp chính để chọn mẫu tiếp theo
    th_iter = 0
    while mask.sum() < budget:
        if budget - mask.sum() >= iter:
            i = iter
        else:
            i = budget - mask.sum()
        k = k_iter
        th_iter += 1
        while i > 0:
            if i < k:
                k = i
            if iter_type == "IDDS":
                sample = select_idds_samples(unlabel_train_data[~mask], labeled_train_data, lamb, k=k, device=device)
            elif iter_type == "random":
                sample = select_random_samples(unlabel_train_data[~mask], k)
            sample["iter"] = th_iter
            mask[sample.index.tolist()] = True
            labeled_train_data = pd.concat([labeled_train_data, sample], ignore_index=True)
            progress_bar.update(k)
            i = i - k
    return labeled_train_data
    

In [5]:
# drive function

def authenticate_service_account(path, scope):
    """Xác thực bằng service account và trả về đối tượng service."""
    try:
        creds = service_account.Credentials.from_service_account_file(
            path, scopes=scope)
        service = build('drive', 'v3', credentials=creds)
        print("Xác thực bằng Service Account thành công!")
        return service
    except Exception as e:
        print(f"Lỗi xác thực: {e}")
        return None

def ls(drive, folder_id):
    if not drive:
        return 'drive is none'
    try: 
        results = drive.files().list(
            q=f'"{folder_id}" in parents',
        ).execute()
        file_list = results.get('files', [])
        minimum = []
        for file in file_list:
            minimum.append(f"{'+' if file['mimeType'].split('.')[-1] == 'folder' else '-'} {file['name']} ({file['id']}) {file['mimeType'].split('.')[-1]}")
        return minimum
    except:
        return 'error'

def download_file(drive, file_id, save_as):
    if not drive:
        return 'drive is none'
    try: 
        request = drive.files().get_media(fileId=file_id)
    
        with open(save_as, "wb") as f: #mo file o che do ghi bin
            downloader = MediaIoBaseDownload(f, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"Da tai xuong {int(status.progress()*100)}%")
        print(f"Tai tep thanh cong, luu tai: {save_as}")
        return True
    except HttpError as error:
        print(f"Loi API: {error}")
        if error.resp.status == 404:
            print("Lỗi 404: tệp không tồn tại hoặc không có quyền truy cập")
        return False
    except IOError as e:
        print(f"Lỗi ghi tệp: {e}")
        return False
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return False
# WARNING:googleapiclient.http:Encountered 403 Forbidden with reason "storageQuotaExceeded"

# Lỗi API: <HttpError 403 when requesting None returned "The user's Drive storage quota has been exceeded.". Details: "[{'message': "The user's Drive storage quota has been exceeded.", 'domain': 'usageLimits', 'reason': 'storageQuotaExceeded'}]">
def upload_file(drive, path, file_name, folder_id=None, overwrite=True):
    full_local_path = os.path.join(path, file_name)
    if not os.path.isfile(full_local_path):
        print(f"Lỗi tệp không tồn tại")
        return None
    ex = file_exists(drive, file_name, folder_id)
    mime_type, _ = mimetypes.guess_type(full_local_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'
    metadata = {
        "name": file_name,
        "parents": [folder_id if folder_id else 'root']
    }
    media = MediaFileUpload(full_local_path, mimetype=mime_type, resumable=True)

    try:
        if ex == None or overwrite == False:
            file = drive.files().create(
                body = metadata,
                media_body=media,
                fields='id, name' # yeu cau tra ve id va name
            ).execute()
            print(f"Tải lên thành công, Id tệp: '{file.get('id')}'")
            return file.get('id')
        else:
            file = drive.files().update(
                fileId=ex['id'],
                media_body=media,
                fields='id, name'
            ).execute()
            print(f"Cập nhật thành công, Id tệp: '{file.get('id')}'")
            return file.get('id')
    except HttpError as e:
        print(f"Lỗi API: {e}")
        decoded_string = e.content.decode('utf-8')
        reason = json.loads(decoded_string)['error']['errors'][0]['reason']
        if reason == 'storageQuotaExceeded':
            drive = authenticate_service_account('service-account-key-2.json', SCOPES)
            try:
                if ex == None or overwrite == False:
                    file = drive.files().create(
                        body = metadata,
                        media_body=media,
                        fields='id, name' # yeu cau tra ve id va name
                    ).execute()
                    print(f"Tải lên thành công, Id tệp: '{file.get('id')}'")
                    return file.get('id')
                else:
                    file = drive.files().update(
                        fileId=ex['id'],
                        media_body=media,
                        fields='id, name'
                    ).execute()
                    print(f"Cập nhật thành công, Id tệp: '{file.get('id')}'")
                    return file.get('id')
            except Exception as e:
                print(f"Lỗi không xác định: {e}")
                return None
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None

def get_id_of_title(drive, title, parent_directory_id='root'):
    try:
        res = drive.files().list(
            q = f"name='{title}' and '{parent_directory_id}' in parents and trashed=false",
            spaces='drive',
            fields='files(id,name)',
            pageSize=1
        ).execute()
        res = res.get('files', [])
        if not res:
            return None
        else:
            return res[0]['id']
    except HttpError as error:
        print(f"Lỗi API: {error}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None

def file_exists(drive, file_name, parent_id):
    try:
        res = drive.files().list(
            q = f"name='{file_name}' and '{parent_id}' in parents and trashed=false",
            spaces='drive',
            fields='files(id,name,mimeType)',
            pageSize=1
        ).execute()
        res = res.get('files', [])
        if not res:
            return None
        else:
            return res[0]
    except HttpError as error:
        print(f"Lỗi API: {error}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None

def create_folder(drive, name, parent_id=None):
    if not drive:
        return None
    if parent_id == None:
        parent_id = 'root'
    try:
        res = drive.files().list(
            q=f"name='{name}' and '{parent_id}' in parents and trashed =false",
            fields='files(id)'
        ).execute()
        res = res.get('files', [])
        if res:
            print("Thư mục đã tồn tại");
            return None
        else:
            metadata = {
                'name': name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_id]
            }
            folder = drive.files().create(
                body=metadata,
                fields='id, name'
            ).execute()
            print(f'Tạo thư mục thành công: {folder.get("id")}')
            return folder.get('id')
    except HttpError as error:
        print(f"Lỗi API: {error}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None
    

def delete_file(drive, file_name, parent_id=None):
    if not drive:
        return None
    try:
        file = drive.files().list(
            q = f"name='{file_name}' and '{parent_id}' in parents and trashed=false",
            pageSize=1,
        ).execute()
        file = file.get('files',[])
    except HttpError as error:
        print(f"Lỗi API khong get duoc id: {error}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None

    try:
        if file == []:
            print("File không tồn tại")
            return None
        file = drive.files().delete(
            fileId=file[0]['id'],
            fields='id'
        ).execute()
        print("Đã xóa file")
        return True
    except HttpError as error:
        print(f"Lỗi API khong xoa duoc file: {error}")
        return None
    except Exception as e:
        print(f"Lỗi không xác định: {e}")
        return None

def download_folder(drive, folder_id, save_as):
    """
    Tải toàn bộ một thư mục và nội dung của nó từ Google Drive.

    Args:
        drive: Đối tượng service đã được xác thực của Google Drive API.
        folder_id (str): ID của thư mục trên Drive cần tải về.
        save_as (str): Đường dẫn đến thư mục trên máy tính
                                      nơi sẽ chứa thư mục tải về.
    """
    try:
        # Lấy tên của thư mục gốc để tạo thư mục tương ứng ở local
        folder_info = drive.files().get(fileId=folder_id, fields='name').execute()
        folder_name = folder_info.get('name')
        
        # Tạo đường dẫn đầy đủ cho thư mục sẽ được tải về
        local_folder_path = os.path.join(save_as, folder_name)
        if os.path.exists(local_folder_path):
            shutil.rmtree(local_folder_path)
        else:
            print(f"📁 Tạo thư mục cục bộ: '{local_folder_path}'")
        os.makedirs(local_folder_path, exist_ok=True)

        # Liệt kê tất cả các mục trong thư mục trên Drive
        query = f"'{folder_id}' in parents and trashed = false"
        results = drive.files().list(q=query,
                                       pageSize=1000, # Lấy tối đa 1000 mục mỗi lần
                                       fields="nextPageToken, files(id, name, mimeType)").execute()
        items = results.get('files', [])

        if not items:
            print(f"   Thư mục '{folder_name}' rỗng.")
            return

        # Dùng tqdm để hiển thị thanh tiến trình
        with tqdm(total=len(items), desc=f'📥 Tải xuống {folder_name}', unit='item') as pbar:
            for item in items:
                item_name = item.get('name')
                item_id = item.get('id')
                item_mime_type = item.get('mimeType')
                
                if item_mime_type == 'application/vnd.google-apps.folder':
                    # Nếu là thư mục, gọi đệ quy để tải thư mục con
                    pbar.set_description(f"📂 Đệ quy vào {item_name}")
                    download_folder(drive, item_id, local_folder_path)
                else:
                    # Nếu là tệp, gọi hàm download_file
                    pbar.set_description(f"📄 Tải tệp {item_name}")
                    local_file_path = os.path.join(local_folder_path, item_name)
                    download_file(drive, item_id, local_file_path)
                
                pbar.update(1)

    except HttpError as error:
        print(f"Lỗi API khi xử lý thư mục ID '{folder_id}': {error}")

# def upload_folder(drive, local_path, parent_folder_id=None, overwrite=False):  
# def delete_folder_recursive(drive, folder_identifier, parent_id=None, include_folder = True):


In [ ]:
#train model
def postprocess_text(preds, labels):
    preds = [[pre.strip() for pre in pred] for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE kì vọng dòng mới sau mỗi câu
    preds = [["\n".join(nltk.sent_tokenize(pre)) for pre in pred] for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels
    
def train_method( **kwargs):
    total_train_data = kwargs.get('total_train_data', None)
    #print(len(total_train_data))
    assert total_train_data is not None  
    result_log = kwargs['df_log']
    log_write = kwargs.get('log_write', None)
    model_name = kwargs.get('model_name',None)
    select_strategy = kwargs.get('select_strategy', None)
    encoder = kwargs.get('encoder', None)
    state_config = kwargs.get('state_config', {})
    alpha = kwargs.get('alpha', 0.0005*100)
    last_test_score = -1
    test_score = 0
    ind = state_config['iter']
    #train model
    bertscore = bertscores
    bleu = evaluate.load("bleu")
    while abs(test_score - last_test_score)*100 > alpha :
        upload_file(drive, WORKING_DIR, 'app.log', folder_id=BACKUP_DIR_ID)
        upload_file(drive, WORKING_DIR, 'app_critical.log', folder_id=BACKUP_DIR_ID)
        last_test_score = test_score
        if ind > max(total_train_data["iter"].unique().tolist()):
            state_config['iter'] = 0
            with open('state_config.json', 'w', encoding='utf-8') as f:
                json.dump(state_config, f, ensure_ascii=False, indent=4)
            upload_file(drive, WORKING_DIR, 'state_config.json', folder_id=BACKUP_DIR_ID)
            break
        
        print(f"\n\n================ITER:{ind}=======================")
        log_write.info(f"ITER:{ind}")
        state_config['iter'] = ind
        with open('state_config.json', 'w', encoding='utf-8') as f:
            json.dump(state_config, f, ensure_ascii=False, indent=4)
        upload_file(drive, WORKING_DIR, 'state_config.json', folder_id=BACKUP_DIR_ID)
        #print("check 3")
        #print(Dataset.from_pandas(total_train_data[ind]))
        kwargs['df_log'] = result_log
        result_log = train_model(
            train_detail_ds = Dataset.from_pandas(total_train_data[total_train_data["iter"] <= ind]),
            iter = ind,
            num_of_samples = len(total_train_data[total_train_data["iter"] <= ind]),
            bert_score = bertscore,
            #meteor_score = meteor,
            bleu_score = bleu,
            **kwargs
        )
        result_log.to_csv("result.csv",index=False)
        upload_file(drive, WORKING_DIR, 'result.csv', folder_id=BACKUP_DIR_ID)
        log_write.info(f"ITER:{ind} Done")
        ind += 1
        
        test_score = result_log.loc[len(result_log)-1, "rouge_avg_score"]
        print(f"different: {test_score} : {last_test_score} : {test_score - last_test_score}")
    if abs(test_score - last_test_score)*100 <= alpha:
        state_config['iter'] = 0
        with open('state_config.json', 'w', encoding='utf-8') as f:
            json.dump(state_config, f, ensure_ascii=False, indent=4)
        upload_file(drive, WORKING_DIR, 'state_config.json', folder_id=BACKUP_DIR_ID)
        upload_file(drive, WORKING_DIR, 'app.log', folder_id=BACKUP_DIR_ID)
        upload_file(drive, WORKING_DIR, 'app_critical.log', folder_id=BACKUP_DIR_ID)
    return result_log

def train_model( **kwargs ):
    train_detail_ds = kwargs.get('train_detail_ds', None)
    val_detail = kwargs.get('val_detail', None)
    test_detail = kwargs.get('test_detail', None)
    model_path = kwargs.get('model_path', None)
    model_name = kwargs.get('model_name',None)
    output_dir = kwargs.get('output_dir', None)
    batch_size = kwargs.get('batch_size', 8)
    num_train_epochs = kwargs.get('num_train_epochs', 3)
    limit_new_tokens = kwargs.get('limit_new_tokens', 512)
    select_strategy = kwargs.get('select_strategy', None)
    encoder = kwargs.get('encoder', None)
    loop = kwargs.get('loop', None)
    device = kwargs.get('device', torch.device("cpu"))
    df_log = kwargs.get('df_log', pd.DataFrame(columns=result_log_template_col))
    iter = kwargs.get('iter', None)
    num_of_samples = kwargs.get('num_of_samples', None)
    log_write = kwargs.get('log_write', None)
    bert_score = kwargs.get('bert_score',bertscores)
    #meteor_score = kwargs.get('meteor_score', evaluate.load('meteor'))
    bleu_score = kwargs.get('bleu_score', evaluate.load('bleu'))
    num_var = kwargs.get('num_var', 1)
    var_certainty = kwargs.get('var_certainty', 'bleu')
    test_score = 0
    last_test_score = test_score
    train_detail_ds.reset_format()
    val_detail.reset_format()
    test_detail.reset_format()

    if "T5" in model_name:
        model_iter = T5ForConditionalGeneration.from_pretrained(model_path)
        model_tokenizer = AutoTokenizer.from_pretrained(model_path)
        print('Use model T5')
    elif re.search("bart", model_name, re.I) != None:  
        model_iter = MBartForConditionalGeneration.from_pretrained(model_path)
        if re.search("mbart", model_name, re.I) != None:
            model_tokenizer = AutoTokenizer.from_pretrained(model_path, src_lang="vi_VN", tgt_lang="vi_VN")
            print('Use model mbart')
        else:
            model_tokenizer = AutoTokenizer.from_pretrained(model_path)
            print('Use model bartpho')
    model_iter.to(device)
    
    tokenized_train_datasets = train_detail_ds.map(
        lambda x:  model_tokenizer(x["raw_text"], text_target=x["anchor_text"] , max_length=max_input_length, truncation=True), 
        remove_columns=['cluster_id', 'title', 'anchor_text', 'raw_text', 'embeddings', 'iter']
    )
    tokenized_val_datasets = val_detail.map(
        lambda x: model_tokenizer(x["raw_text"], text_target=x["anchor_text"] , max_length=max_input_length, truncation=True),
        remove_columns=['cluster_id', 'title', 'anchor_text', 'raw_text'] 
    )
    #tokenized_test_datasets = test_detail.map(lambda x: preprocess_function(x,model_tokenizer), remove_columns=['cluster_id', 'title', 'anchor_text', 'raw_text'] )
    tokenized_train_datasets.set_format("torch")
    tokenized_val_datasets.set_format("torch")
    #tokenized_test_datasets.set_format("torch") 
    
    #model_iter = MBartForConditionalGeneration.from_pretrained("/kaggle/working/origin_backup/bartpho")
    data_collator = DataCollatorForSeq2Seq(model_tokenizer, model=model_iter)
    print(f"train_detail_ds : {len(train_detail_ds)}")
    train_dataloader = DataLoader(
        tokenized_train_datasets, shuffle=True, collate_fn=data_collator, batch_size=batch_size,
    )
    eval_dataloader = DataLoader(
        tokenized_val_datasets, shuffle=True, collate_fn=data_collator, batch_size=batch_size
    )
    # test_dataloader = DataLoader(
    #     tokenized_test_datasets, shuffle=True, collate_fn=data_collator, batch_size=batch_size,
    # )
    optimizer = AdamW(model_iter.parameters(), lr=2e-5)
    accelerator = Accelerator(cpu = 'cpu' in device.type)
    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model_iter, optimizer, train_dataloader, eval_dataloader
    )


    num_update_steps_per_epoch = len(train_dataloader)
    num_training_steps = num_train_epochs * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    rouge_score = evaluate.load("rouge")
    log_list = []
    log_dict = {
        "Dataset" : "abmusu",
        "Encoder" : encoder,
        "Model" : model_name,
        "Select strategy" : select_strategy,
        "Loop": loop,
        "Iter" : iter,
        "Num of Samples": num_of_samples,
        "Epoch" : 0,
        "Type score" : "",
        "rouge_avg_score" : 0.0, 
        "rouge1" : 0.0, 
        "rouge2" : 0.0, 
        "rougeL" : 0.0, 
        "rougeLsum" : 0.0, 
        "bleu" : 0.0, 
        "bleu_std" : 0.0, 
        "bleu_1" : 0.0, 
        "bleu_2" : 0.0,
        "bleu_3" : 0.0, 
        "bleu_4" : 0.0,
        "bleu_brevity_penalty" : 0.0, 
        "bleu_length_ratio" : 0.0, 
        "bleu_translation_length" : 0, 
        "bert_precision" : 0.0, 
        "bert_recall" : 0.0, 
        "bert_f1" : 0.0,
        "bert_std" : 0.0,
        "Time" : 0,
    }
    
    progress_bar = tqdm(range(num_training_steps), desc="Train")
    model.train()
    for epoch in range(num_train_epochs):
        time_start = time.time()
        # Huấn luyện
        for step, batch in enumerate(train_dataloader):
            batchs = {
                'input_ids' : batch['input_ids'],#.to(cudadevice), 
                'attention_mask' : batch['attention_mask'],#.to(cudadevice), 
                'labels' : batch['labels'],#.to(cudadevice)
            }
            outputs = model(**batchs)
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
        
        # Đánh giá epoch
        log_write.info(f"train epoch {epoch} done")
        #print("check1")
    model.eval()
    predictions_list = []
    certain_list = []
    labels_list = []
    bert_std_list = []
    bleu_std_list = []
    for module in model.modules():
        if module.__class__.__name__.startswith('Dropout'):
            module.train()
    eval_progress_bar = tqdm(range(len(eval_dataloader)), desc="Eval")
    if num_var < 1:
        num_var = 1
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            if num_var == 1:
                generated_tokens = accelerator.unwrap_model(model).generate(
                    batch["input_ids"],#.to(cudadevice),
                    attention_mask=batch["attention_mask"],#.to(cudadevice),
                    max_new_tokens = limit_new_tokens,
                )
            else:
                num_var = int(num_var)
                generated_tokens = accelerator.unwrap_model(model).generate(
                    batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    max_new_tokens = limit_new_tokens,
                    num_return_sequences=num_var,
                    do_sample=True,
                    top_k=50,
                    top_p=0.95
                )
            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=model_tokenizer.pad_token_id
            )
            labels = batch["labels"]
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=model_tokenizer.pad_token_id
            )
            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            generated_tokens = generated_tokens.reshape(len(batch["input_ids"]), num_var, -1)
            labels = accelerator.gather(labels).cpu().numpy()
            labels = np.where(labels != -100, labels, model_tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = [model_tokenizer.batch_decode(generated_token, skip_special_tokens=True) for generated_token in generated_tokens]
            decoded_labels = model_tokenizer.batch_decode(labels, skip_special_tokens=True)
            decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

            predictions_list.extend(decoded_preds)
            labels_list.extend(decoded_labels)
            eval_progress_bar.update(1)
            del generated_tokens
            del labels
    if num_var > 1:
        for l in range(len(predictions_list)):
            f = 0
            while f < len(predictions_list[l]):
                if predictions_list[l][f] == '':
                    #decoded_preds[l].pop(f)
                    predictions_list[l][f] = 'a'
                else:
                    f += 1
        for l in range(len(predictions_list)):
            bleu_matrix = [[bleu_score.compute(predictions=[i], references=[j])['bleu'] for j in predictions_list[l]] for i in predictions_list[l]]
            bleu_std_list.append(np.std([bleu_matrix[i][j] for i in range(0, len(predictions_list[l])-1) for j in range(i+1, len(predictions_list[l]))]))
            if var_certainty == 'bleu':
                certain_list.append(predictions_list[l][np.argmax(np.sum(bleu_matrix, axis = 1))])
        for l in range(len(predictions_list)):
            pred_for_bert = [predictions_list[l][i] for i in range(0, len(predictions_list[l])-1) for j in range(i+1, len(predictions_list[l]))]
            ref_for_bert = [predictions_list[l][j] for i in range(0, len(predictions_list[l])-1) for j in range(i+1, len(predictions_list[l]))]
            bert_f1 = bert_score(pred_for_bert, ref_for_bert, model_type="bert-base-multilingual-cased")[2]
            bert_matrix = [[0.0] * len(predictions_list[l])] * len(predictions_list[l])
            for i in range(0, len(predictions_list[l])):
                for j in range(0, len(predictions_list[l])):
                    if i < j:
                        bert_matrix[i][j] = bert_f1[i*len(predictions_list[l]) + j - 1 - (i+3)*i//2]
                    elif i > j:
                        bert_matrix[i][j] = bert_f1[j*len(predictions_list[l]) + i - 1 - (j+3)*j//2]
                    else:
                        bert_matrix[i][j] = torch.tensor(1.0)
            bert_std_list.append(np.std(bert_f1.tolist()))
            #last list to cal rouge, bleu, bert_f1
            if var_certainty == 'bert':
                certain_list.append(predictions_list[l][np.argmax(np.sum(bert_matrix, axis = 1))])
            # certain_list.append(decoded_preds[l][0])
    else:
        for l in range(len(predictions_list)):
            if predictions_list[l][0] == '':
                predictions_list[l][0] = 'a'
            bleu_std_list.append(1)
            bert_std_list.append(1)
            certain_list.append(predictions_list[l][0])
    # Tính toán các chỉ số

    result_rouge_score = rouge_score.compute(predictions=certain_list, references=labels_list)
    result_rouge_score = {key: round(value * 100,4) for key, value in result_rouge_score.items()}
    #result_meteor_score = meteor.compute(predictions=predictions, references=references)
    try:
        result_bleu_score = bleu_score.compute(predictions=certain_list, references=labels_list)
    except:
        result_bleu_score = -1
    bert_precision, bert_recall, bert_f1 = bert_score(certain_list, labels_list, model_type="working/origin_backup/bert-base-multilingual-cased", num_layers=9,  lang="vi", verbose=True)
    bert_precision = round(float(bert_precision.mean())*100,4)
    bert_recall = round(float(bert_recall.mean())*100,4)
    bert_f1 = round(float(bert_f1.mean())*100,4)
    bert_std = round(np.average(bert_std_list)*100,4)
    bleu_std = np.average(bleu_std_list)
    test_score = sum(result_rouge_score.values()) / 4
    te = log_dict.copy()
    te['Type score'] = 'Val'
    te['Time'] = time.time() - time_start
    te['rouge_avg_score'] = test_score
    te['Epoch'] = epoch
    te['rouge1'] = result_rouge_score['rouge1']
    te['rouge2'] = result_rouge_score['rouge2']
    te['rougeL'] = result_rouge_score['rougeL']
    te['rougeLsum'] = result_rouge_score['rougeLsum']
    # te['meteor'] = round(result_meteor_score['meteor'] * 100,4)
    te['bleu'] = result_bleu_score['bleu']
    te['bleu_std'] = bleu_std
    te['bleu_1'] = result_bleu_score['precisions'][0]
    te['bleu_2'] = result_bleu_score['precisions'][1]
    te['bleu_3'] = result_bleu_score['precisions'][2]
    te['bleu_4'] = result_bleu_score['precisions'][3]
    te['bleu_brevity_penalty'] = result_bleu_score['brevity_penalty']
    te['bleu_length_ratio'] = result_bleu_score['length_ratio']
    te['bleu_translation_length'] = result_bleu_score['translation_length']
    te['bert_precision'] = bert_precision
    te['bert_recall'] = bert_recall
    te['bert_f1'] = bert_f1
    te['bert_std'] = bert_std
    
    log_list.append(te)

    
    # print(f"Epoch {epoch}:", result_rouge_score)
    # print(f"different: {test_score} : {last_test_score} : {test_score - last_test_score}")
    # last_test_score = test_score
    log_write.info(f"test done")
    # Lưu và tải
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        model_tokenizer.save_pretrained(output_dir)
    
    #test iter
    # print(f"last_test_score: {last_test_score}")
    print(f"test_score: {test_score}")

    new_log_df = pd.DataFrame(log_list)
    del model_iter
    del model_tokenizer
    del train_dataloader
    del eval_dataloader
    # del test_dataloader
    torch.cuda.empty_cache()
    if len(df_log) == 0:
        return new_log_df
    else:
        return pd.concat([df_log, new_log_df], axis=0,ignore_index=True)
    #Run test

In [7]:
# GPU function
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_reserved()/1024**2)
def memory_cache_clear():
    torch.cuda.empty_cache()
    memory_stats()

In [8]:
max_input_length = 1024
max_target_length = 1024

MODE = "RUN"

WORKING_DIR = '.'
if MODE == "TEST":
    BACKUP_DIR_ID = '1rhuFAQnmGBoVuARyhpcX8f_LDjcEHCGB'
else:
    BACKUP_DIR_ID = '15i-as_VUYJBRXXie01FgUZo9AqDiq-Cn'

SERVICE_ACCOUNT_FILE = 'service-account-key-4.json' # Đường dẫn tới tệp key của service account
SCOPES = ['https://www.googleapis.com/auth/drive'] # Phạm vi truy cập. Vẫn cần thiết để định nghĩa quyền hạn.

CUDA_INDEX = 6

In [9]:
# setup template state config
template_state_config = {}
if MODE == "TEST":
    #just for test
    template_state_config['loop'] = 1 # measure stability
    template_state_config['iter_init'] = 8
    template_state_config['iter_num'] = 5
    template_state_config['select_lamb'] = 0.67
    template_state_config['num_train_epochs'] = 2
    template_state_config['alpha'] = 0.005*100
    template_state_config['limit_new_tokens'] =512
    template_state_config['batch_size'] = 2 #bartpho dung 16, mbart chi duoc 2, viT5:4 ok với 6iter, mT5:ko xai duoc
    template_state_config['num_var'] = 2
    template_state_config['var_certainty'] = 'bert' # hoặc 'bleu'
  
else:
    #for real
    template_state_config['loop'] = 3
    template_state_config['iter_init'] = 100
    template_state_config['iter_num'] = 50
    template_state_config['select_lamb'] = 0.67
    template_state_config['num_train_epochs'] = 10 # 10
    template_state_config['alpha'] = 0.005*100
    template_state_config['limit_new_tokens'] = 512
    template_state_config['batch_size'] = 8
    template_state_config['num_var'] = 1
    template_state_config['var_certainty'] = 'bert' # hoặc 'bleu'
    # vit5 24 tầm 68.33GB it6 -> 78.54GB+ it8(hẹo), 
    # vit5 20 tầm 61GB
    # vit5 16 tầm 46GB -> 47.44 ở iter 5, 
    # vit5 12 tầm 37.73GB(it8)->41.69(it13)
    # vit5 8  tầm 27.8GB, 
    # vit5 4  tầm 

    # bartpho 24 tầm 21.15GB
    # bartpho 32 tầm 31.18GB -> 36.23GB(it12)
    # bartpho 48 tầm 50.15GB(i6L0) -> 52.36GB(i7L1)

    # mT5 16 crash
    # mT5 12 67.62GB(i0) -> 77.67GB(i8) -> crash(it13L1) ~ 83GB
    # mT5 10 74.15GB(i13) -> 78GB(it16)

    # mbart 36 71.99GB(i0L0) -> 76.30(i4L0) -> 78.31GB(i6L0) -> crash(it10) ~ 85GB
    # mbart 32 74.86GB(i10L0) -> 78.15GB(i0L1) -> 83.15GB(i14L1)
    # mbart 24 76.66GB(i14L1)
template_state_config['name'] = 'train model mt5 budget=full loop=3 numvar=1 certainty=bert'
template_state_config['description'] = 'train model với quy trình cập nhật có lấy cả certainty'
template_state_config['budget'] = 1200
template_state_config['current_loop'] = 0
template_state_config['iter'] = 0
template_state_config['output_dir'] = WORKING_DIR
template_state_config['encoders'] = ['phobert', 'bert-base-multilingual-cased']
template_state_config['current_encoder'] = template_state_config['encoders'][0]
template_state_config['models'] = ['mT5']
template_state_config['current_models'] = template_state_config['models'][0]
template_state_config['select_strategies'] = ["IDDS", "random"]
template_state_config['current_select_strategies'] = template_state_config['select_strategies'][0]
#config for embedding
template_state_config['graph_split'] = None               # truncate - None / sentence split to sentences
template_state_config['graph_merge'] = None              # mean/max/None pooling
template_state_config['sentence_split'] = None            # truncate - None / split to 2 sentence, default truncate
template_state_config['sentence_merge'] = None            # mean/max/None pooling
template_state_config['sentence_vec'] = None             # cls - None/mean/max pooling


result_log_template_col = ["Dataset", "Encoder", "Model", "Select strategy", "Loop", "Iter", "Num of Samples", "Epoch", "Type score", 
                            "rouge_avg_score", "rouge1", "rouge2", "rougeL", "rougeLsum", 
                            "bleu", "bleu_std", "bleu_1", "bleu_2", "bleu_3", "bleu_4", "bleu_brevity_penalty", "bleu_length_ratio", "bleu_translation_length", 
                            "bert_precision", "bert_recall", "bert_f1", "bert_std", "Time"]

In [10]:
drive = authenticate_service_account(SERVICE_ACCOUNT_FILE, SCOPES)

Xác thực bằng Service Account thành công!


In [11]:
state_exist = file_exists(drive, 'state_config.json', BACKUP_DIR_ID)
if state_exist != None:
    download_file(drive, state_exist['id'], WORKING_DIR+'/state_config.json')
    with open(WORKING_DIR+'/state_config.json', 'r') as f:
        state_config = json.load(f)
else: 
    state_config = template_state_config

result_log_exist = file_exists(drive, 'result.csv', BACKUP_DIR_ID)
if result_log_exist != None:
    download_file(drive, result_log_exist['id'], WORKING_DIR+'/result.csv')
    result_log = pd.read_csv(WORKING_DIR+'/result.csv')
else:
    result_log = pd.DataFrame(columns=result_log_template_col)

data_backup_exist = file_exists(drive, f'data_backup_model-{state_config["current_models"]}_strategies-{state_config["current_select_strategies"]}_encoder-{state_config["current_encoder"]}_loop-{state_config["current_loop"]}.jsonl', BACKUP_DIR_ID)
if data_backup_exist != None:
    download_file(drive, data_backup_exist['id'], WORKING_DIR+f'/working/data/data_backup_model-{state_config["current_models"]}_strategies-{state_config["current_select_strategies"]}_encoder-{state_config["current_encoder"]}_loop-{state_config["current_loop"]}.jsonl')

app_exist = file_exists(drive, 'app.log', BACKUP_DIR_ID)
if app_exist != None:
    download_file(drive, app_exist['id'], WORKING_DIR+'/app.log')

app_critical_exist = file_exists(drive, 'app_critical.log', BACKUP_DIR_ID)
if app_critical_exist != None:
    download_file(drive, app_critical_exist['id'], WORKING_DIR+'/app_critical.log')
    
logger = initialize_logging(__name__)
logger.info('authenticate service account successful')
logger.info(f"{MODE} mode")

Da tai xuong 100%
Tai tep thanh cong, luu tai: ./state_config.json
Da tai xuong 100%
Tai tep thanh cong, luu tai: ./result.csv
Da tai xuong 100%
Tai tep thanh cong, luu tai: ./working/data/data_backup_model-mT5_strategies-IDDS_encoder-bert-base-multilingual-cased_loop-1.jsonl
Da tai xuong 100%
Tai tep thanh cong, luu tai: ./app.log


[2025-07-06 18:03:47.995 - DEBUG]  initialized
[2025-07-06 18:03:47.996 - INFO]  authenticate service account successful
[2025-07-06 18:03:47.998 - INFO]  RUN mode


Da tai xuong 0%
Tai tep thanh cong, luu tai: ./app_critical.log


In [12]:
#config device
torch.cuda.set_device(CUDA_INDEX)
cudadevice = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
cpudevice = torch.device("cpu")

# if not os.path.exists('./working/origin_backup/phobert'):
#     download_folder(drive, '1YXkURCQp7aUUe0pskKzMufidofCnwV-Z', './working/origin_backup/phobert')
# if not os.path.exists('./working/origin_backup/viT5'):
#     download_folder(drive, '19o4wxqTTeP8owtV6Kthbxw69k20FPEWq', './working/origin_backup/viT5')
#Load model encoder
# checkpoint_encoder = "google-bert/bert-base-multilingual-cased"
# checkpoint_encoder = 'keepitreal/vietnamese-sbert'
# checkpoint_encoder = 'vinai/phobert-base-v2'
# local_checkpoint_encoder = f'./working/origin_backup/{state_config["current_encoder"]}'
# tokenizer_encoder = AutoTokenizer.from_pretrained(local_checkpoint_encoder, local_files_only=True)
# model_encoder = AutoModel.from_pretrained(local_checkpoint_encoder, local_files_only=True)
# encoder_seq_length = 512
# model_encoder.to(cudadevice)
# logger.info(f"{state_config['current_encoder']} cloned")

#Load model main
# checkpoint_bartpho = "vinai/bartpho-word-base"
# bartpho_tokenizer = AutoTokenizer.from_pretrained(checkpoint_bartpho)
# bartpho_model = BartForConditionalGeneration.from_pretrained(checkpoint_bartpho)
# bartpho_model.save_pretrained("/kaggle/working/origin_backup/bartpho")
# bartpho_tokenizer.save_pretrained("/kaggle/working/origin_backup/bartpho")
# #bartpho_model.to(cudadevice);
# logger.info("bartpho-word-base cloned")

# checkpoint_mbart = "facebook/mbart-large-50" # 680M params
# mbart_tokenizer = AutoTokenizer.from_pretrained(checkpoint_mbart)
# mbart_model = MBartForConditionalGeneration.from_pretrained(checkpoint_mbart)
# mbart_model.save_pretrained("./working/origin_backup/mbart")
# mbart_tokenizer.save_pretrained("./working/origin_backup/mbart")
# #mbart_model.to(cudadevice);
# logger.info("mbart-large-50 cloned")

# checkpoint_viT5 = "VietAI/vit5-base" # 310M params and 866M params for large version
# viT5_tokenizer = AutoTokenizer.from_pretrained(checkpoint_viT5, force_download=True)
# viT5_model = T5ForConditionalGeneration.from_pretrained(checkpoint_viT5, force_download=True)
# viT5_model.save_pretrained("./working/origin_backup/viT5")
# viT5_tokenizer.save_pretrained("./working/origin_backup/viT5")
# #mbart_model.to(cudadevice);
# logger.info("viT5-base cloned")


#mbart_model.to(cudadevice);
# logger.info("viT5-base cloned")

# checkpoint_mT5 = "google/mt5-base" # 580 million parameters
# mT5_tokenizer = AutoTokenizer.from_pretrained(checkpoint_mT5)
# mT5_model = T5ForConditionalGeneration.from_pretrained(checkpoint_mT5)
# mT5_model.save_pretrained("./working/origin_backup/mT5")
# mT5_tokenizer.save_pretrained("./working/origin_backup/mT5")
# #mbart_model.to(cudadevice);
# logger.info("mT5-base cloned")



In [13]:
#preprocess abmusu
#limit data train to test
if MODE == "TEST":
    num_sample_train = 16
    num_sample_val = 16
    num_sample_test = 8
else:
    num_sample_train = -1
    num_sample_val = -1
    num_sample_test = -1


#Load data abmusu
# data_path = {"train":"abmusu/vlsp_2022_abmusu_train_data_new.jsonl",
#              "val":"abmusu/vlsp_2022_abmusu_validation_data_new.jsonl",
#              "test":"abmusu/vlsp_abmusu_test_data.jsonl"
#             }
# data = load_dataset("json", data_files=data_path)
# train_df_detail_ex = pd.read_csv('abmusu/abmusu_1200.csv')
# display(data)

# if MODE == "TEST":
#     #just for test
#     train_df_detail, train_df_cluster_info = preprocess_dataset(data["train"].select(range(num_sample_train)))
#     train_df_detail = train_df_detail[:num_sample_train]
#     val_df_detail, val_df_cluster_info = preprocess_dataset(data["val"].select(range(num_sample_val)))
#     val_df_detail = val_df_detail[:num_sample_val]
#     test_df_detail, test_df_cluster_info = preprocess_dataset(data["test"].select(range(num_sample_val)))
#     test_df_detail = test_df_detail[:num_sample_val]

#     train_df_detail_ex = train_df_detail_ex[:num_sample_train]
# else:
#     #for real
#     #split to paragraph
#     train_df_detail, train_df_cluster_info = preprocess_dataset(data["train"])
#     val_df_detail, val_df_cluster_info = preprocess_dataset(data["val"])
#     test_df_detail, test_df_cluster_info = preprocess_dataset(data["test"])

In [14]:
# train_detail = Dataset.from_pandas(train_df_detail) # train_df_detail_ex
# val_detail = Dataset.from_pandas(val_df_detail)
# test_detail = Dataset.from_pandas(test_df_detail)

# logger.info('abmusu processed')
# # tach theo cau, theo doan
# # khongg xu ly(mac dinh truncate), truncate
# # sau embedding max,mean pooling, cls theo cau, doan
# train_detail = get_embeddings(
#     train_detail,
#     'raw_text',
#     tokenizer_encoder,
#     model_encoder,
#     encoder_seq_length,
#     device = cudadevice,
#     return_device = cpudevice,
#     graph_split = state_config['graph_split'],                  # truncate - None / sentence split to sentences
#     graph_merge = state_config['graph_merge'],                  # mean/max/None pooling
#     sentence_split = state_config['sentence_split'],            # truncate - None / split to 2 sentence, default truncate
#     sentence_merge = state_config['sentence_merge'],            # mean/max/None pooling
#     sentence_vec = state_config['sentence_vec'],                # cls - None/mean/max pooling
#     return_type = 'datasets',
#     batch_size = 256
# )

# train_detail.add_faiss_index(column="embeddings")
# t = model_encoder.to(cpudevice)
# del model_encoder
# model_encoder = t
# torch.cuda.empty_cache()
# train_detail.features
# logger.info("abmusu embedded")


In [ ]:
encoders = state_config['encoders'][state_config['encoders'].index(state_config['current_encoder']):]
for encoder in encoders:
    models = state_config['models'][state_config['models'].index(state_config['current_models']):]
    for j in models:
        strategies = state_config['select_strategies'][state_config['select_strategies'].index(state_config['current_select_strategies']):]
        for i in strategies:
            current_loop = state_config['current_loop']
            for l in range(current_loop, state_config['loop']):
                logger.info(f"Train {j} - {i} - {encoder} - with th-loop: {l}")
                state_config['current_loop'] = l
                state_config['current_models'] = j
                state_config['current_select_strategies'] = i
                state_config['current_encoder'] = encoder
                with open('state_config.json', 'w', encoding='utf-8') as f:
                    json.dump(state_config, f, ensure_ascii=False, indent=4)
                upload_file(drive, WORKING_DIR, 'state_config.json', folder_id= BACKUP_DIR_ID)
                if os.path.exists(WORKING_DIR+f'/working/data/data_backup_model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}.jsonl') == False or state_config['iter'] == 0:
                    # process data
                    local_checkpoint_encoder = f'./working/origin_backup/{state_config["current_encoder"]}'
                    tokenizer_encoder = AutoTokenizer.from_pretrained(local_checkpoint_encoder, local_files_only=True)
                    model_encoder = AutoModel.from_pretrained(local_checkpoint_encoder, local_files_only=True)
                    encoder_seq_length = int(model_encoder.config.max_position_embeddings / 256) * 256
                    model_encoder.to(cudadevice)
                    logger.info(f"{state_config['current_encoder']} loaded")
                    if MODE == "TEST":
                        train_df_detail = pd.read_csv('../abmusu/abmusu_1200.csv').loc[:num_sample_train,]
                        val_df_detail = pd.read_csv('../abmusu/abmusu_val.csv').loc[:num_sample_val, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                        test_df_detail = pd.read_csv('../abmusu/abmusu_test.csv').loc[:num_sample_test, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                    else:
                        train_df_detail = pd.read_csv('../abmusu/abmusu_1200.csv')
                        val_df_detail = pd.read_csv('../abmusu/abmusu_val.csv').loc[:, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                        test_df_detail = pd.read_csv('../abmusu/abmusu_test.csv').loc[:, ['cluster_id', 'title', 'anchor_text', 'raw_text']]

                    train_detail = Dataset.from_pandas(train_df_detail) # train_df_detail_ex
                    val_detail = Dataset.from_pandas(val_df_detail)
                    test_detail = Dataset.from_pandas(test_df_detail)
                    logger.info('abmusu processed')
                    
                    train_detail = get_embeddings(
                        train_detail,
                        'raw_text',
                        tokenizer_encoder,
                        model_encoder,
                        encoder_seq_length,
                        device = cudadevice,
                        return_device = cpudevice,
                        graph_split = state_config['graph_split'],                  # truncate - None / sentence split to sentences
                        graph_merge = state_config['graph_merge'],                  # mean/max/None pooling
                        sentence_split = state_config['sentence_split'],            # truncate - None / split to 2 sentence, default truncate
                        sentence_merge = state_config['sentence_merge'],            # mean/max/None pooling
                        sentence_vec = state_config['sentence_vec'],                # cls - None/mean/max pooling
                        return_type = 'datasets',
                        batch_size = 256
                    )
                    t = model_encoder.to(cpudevice)
                    del model_encoder
                    model_encoder = t
                    torch.cuda.empty_cache()
                    logger.info("abmusu embedded")
                    
                    train_detail.set_format("pandas")
                    unlabel_train= train_detail[:]

                    total_train_data = data_select(
                        unlabel_train.copy(), budget = state_config['budget'],
                        init = state_config['iter_init'], k_init = -1, init_type = f"{i}",
                        iter = state_config['iter_num'],  k_iter = -1, iter_type = f"{i}",
                        lamb = state_config['select_lamb'], device=cudadevice
                    )
                    data_backup = Dataset.from_pandas(total_train_data)
                    data_backup.to_json(f'working/data/data_backup_model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}.jsonl')
                    upload_file(drive, WORKING_DIR + f'/working/data/', f'data_backup_model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}.jsonl', folder_id=BACKUP_DIR_ID)
                    logger.info(f"backup data DONE")
                else:
                    data = load_dataset("json", data_files=WORKING_DIR+f'/working/data/data_backup_model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}.jsonl')
                    data = data['train']
                    data.set_format("pandas")
                    total_train_data= data[:]
                    if MODE == "TEST":
                        val_df_detail = pd.read_csv('../abmusu/abmusu_val.csv').loc[:num_sample_val, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                        test_df_detail = pd.read_csv('../abmusu/abmusu_test.csv').loc[:num_sample_test, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                    else:
                        val_df_detail = pd.read_csv('../abmusu/abmusu_val.csv').loc[:, ['cluster_id', 'title', 'anchor_text', 'raw_text']]
                        test_df_detail = pd.read_csv('../abmusu/abmusu_test.csv').loc[:, ['cluster_id', 'title', 'anchor_text', 'raw_text']]

                    val_detail = Dataset.from_pandas(val_df_detail)
                    test_detail = Dataset.from_pandas(test_df_detail)
                result_log = train_method(
                    total_train_data = total_train_data,
                    val_detail = val_detail,
                    test_detail = test_detail,
                    encoder = encoder,
                    alpha = state_config['alpha'],
                    model_path = f'{WORKING_DIR}/working/origin_backup/{j}',
                    output_dir = f'{WORKING_DIR}/working/{j}/{i}',
                    batch_size = state_config['batch_size'],
                    model_name = j,
                    num_train_epochs = state_config['num_train_epochs'],
                    limit_new_tokens = state_config['limit_new_tokens'],
                    select_strategy = i,
                    loop = l,
                    state_config = state_config,
                    df_log = result_log,
                    log_write = logger,
                    num_var = state_config['num_var'],
                    var_certainty = state_config['var_certainty'],
                    device = cudadevice
                )
                result_log.to_csv("result.csv",index=False)
                upload_file(drive, WORKING_DIR, 'result.csv', folder_id=BACKUP_DIR_ID)
                model_id = create_folder(drive, f'backup-model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}', parent_id=BACKUP_DIR_ID)
                if model_id:
                    for f in os.listdir(f'{WORKING_DIR}/working/{j}/{i}'):
                        upload_file(drive, f'{WORKING_DIR}/working/{j}/{i}', f, folder_id=model_id)
                else:
                    print('Không thể tạo folder backup model')
                    logger.info(f"Không thể tạo backup-model-{j}_strategies-{i}_encoder-{encoder}_loop-{l}")
            state_config['current_loop'] = 0
        state_config['current_select_strategies'] = state_config['select_strategies'][0]
    state_config['current_models'] = state_config['models'][0]
result_log

In [ ]:
result_log

,Dataset,Encoder,Model,Select strategy,Loop,Iter,Num of Samples,Epoch,Type score,rouge_avg_score,...,bleu_3,bleu_4,bleu_brevity_penalty,bleu_length_ratio,bleu_translation_length,bert_precision,bert_recall,bert_f1,bert_std,Time
0,abmusu,phobert,viT5,IDDS,0,0,8,1,Val,22.779075,...,0.006269,0.002331,1.0,4.888514,4341,57.2434,65.9305,61.2496,0.0,95.527842
1,abmusu,phobert,viT5,IDDS,0,1,13,1,Val,21.027725,...,0.008356,0.003774,1.0,5.429054,4821,58.9836,67.5065,62.9328,0.0,88.575020
2,abmusu,phobert,viT5,IDDS,0,2,17,1,Val,21.196200,...,0.010158,0.005097,1.0,5.581081,4956,58.1596,67.4309,62.4315,0.0,83.347652
3,abmusu,phobert,viT5,random,0,0,8,1,Val,22.960800,...,0.011296,0.005323,1.0,4.923423,4372,58.8391,67.4589,62.8187,0.0,83.235364
4,abmusu,phobert,viT5,random,0,1,13,1,Val,22.957000,...,0.006676,0.002234,1.0,5.099099,4528,58.5039,67.1173,62.4848,0.0,82.989027
5,abmusu,bert-base-multilingual-cased,viT5,IDDS,0,0,8,1,Val,23.285150,...,0.008155,0.002649,1.0,4.733108,4203,58.2752,67.0827,62.3464,0.0,84.417964
6,abmusu,bert-base-multilingual-cased,viT5,IDDS,0,1,13,1,Val,21.021650,...,0.006163,0.002394,1.0,5.702703,5064,57.9655,67.4218,62.3087,0.0,88.572415
7,abmusu,bert-base-multilingual-cased,viT5,IDDS,0,2,17,1,Val,20.917550,...,0.008347,0.003589,1.0,5.704955,5066,59.1320,67.2237,62.8820,0.0,88.054230
8,abmusu,bert-base-multilingual-cased,viT5,random,0,0,8,1,Val,21.718375,...,0.003377,0.000904,1.0,5.040541,4476,57.6373,66.1630,61.5894,0.0,82.067327
9,abmusu,bert-base-multilingual-cased,viT5,random,0,1,13,1,Val,20.065750,...,0.009709,0.004778,1.0,6.185811,5493,58.7372,68.3151,63.1406,0.0,89.041931


In [ ]:
# https://www.youtube.com/watch?v=g_j6ILT-X0k

email_sender = 'minhtuanhth811@gmail.com'
email_password = 'kdho gpnh qsbp gewf'
email_receiver = 'minhtuanhth811@gmail.com'

subject = 'Check Active Learning runtime'
body = f"""
Active Learning đã chạy xong, vui lòng vào check
{state_config['name']}
"""

em = EmailMessage()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = subject
em.set_content(body)

context = ssl.create_default_context()
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receiver, em.as_string())

# Not Use

In [ ]:
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)
def add_embeddings_old(train_data):
    remove_folder_contents('/kaggle/working')
    for i in range(len(train_data)):
        for j in range(len(train_data["single_documents"][i])):
            train_data["single_documents"][i][j]["embeddings"] = get_embeddings(train_data["single_documents"][i][j]["raw_text"]).detach().cpu().numpy()
        train_data.to_csv(f"train_embedding_{i}")
        if os.path.exists(f"/kaggle/working/train_embedding_{i-1}"):
            os.remove(f"/kaggle/working/train_embedding_{i-1}")
        print(f"added embedding to cluster: {i} - category: {train_data['category'][i]}") 

In [ ]:

# cần xử lý trùng title
# xử lý thay title nếu trống anchor text
# có đoạn check duplicate khá hay
# train_df_detail_ex = pd.read_csv('abmusu_1200.csv')
# for i in range(len(train_df_detail_ex)):
#     if pd.isnull(train_df_detail_ex.loc[i,'anchor_text']):
#         print(i)
#         train_df_detail_ex.loc[i,'anchor_text'] = train_df_detail_ex.loc[i,'title']
# c = train_df_detail_ex[train_df_detail_ex['title'].isin(train_df_detail_ex['title'][train_df_detail_ex['title'].duplicated()])].sort_values("title").index.tolist()
# i, ind, j, l = 0, 1, 1, len(c)
# while j < l:
#     if train_df_detail_ex.loc[c[i],'title'] == train_df_detail_ex.loc[c[j],'title']:
#         train_df_detail_ex.loc[c[j],'title'] = train_df_detail_ex.loc[c[j],'title'] + f" ({ind})"
#         ind += 1
#     else:
#         i = j
#         ind = 1
#     j += 1
# train_df_detail_ex.to_csv('abmusu/abmusu_1200.csv')

In [ ]:
#os.remove('/kaggle/working/state_config.json')

In [ ]:
# ls(drive, '1MLkRuZw53k862qGHr9JpPmHIFNd-G0cC')

In [ ]:
# download_file(drive, '12FdkHcqVrdmu5Ae1W5ysPcq0OtulCEpX', '/kaggle/working/result.csv')

In [ ]:
# upload_file(drive, '/kaggle/working', 'result.csv', folder_id='1MLkRuZw53k862qGHr9JpPmHIFNd-G0cC', overwrite=True)